In [1]:
import torch
import matplotlib.pyplot as plt
from torch import optim, distributions, nn
from tqdm.notebook import tqdm
from sklearn.datasets import fetch_olivetti_faces
from gpzoo.kernels import NSF_RBF
from gpzoo.gp import NSF

In [2]:
new_model=True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
data, _ = fetch_olivetti_faces(return_X_y=True, shuffle=True, random_state=97)

In [4]:
Y = (data)*255
Y = Y.astype(int)
Y = torch.tensor(Y)

In [5]:
idx = torch.linspace(-32, 32, 64, dtype=torch.float)
X = torch.cartesian_prod(-idx, idx).flip(dims=[1])
N = X.shape[0]

In [6]:
L =10
M = 32*32
kernel = NSF_RBF(L=L, lengthscale=2)
model = NSF(X=X, y=Y, kernel=kernel, M=M, L=L, jitter=1e-2)
idz = torch.multinomial(torch.ones(X.shape[0]), num_samples=M, replacement=False)
model.svgp.Z = nn.Parameter(X[idz])

if not new_model:
    model.load_state_dict(torch.load('eigenfaces_nsf', map_location=device))
    
model.to(device)

NSF(
  (kernel): NSF_RBF()
  (svgp): SVGP(
    (kernel): NSF_RBF()
  )
)

In [7]:
X_train = X.to(device)
Y_train = Y.to(device)

In [14]:
def train(model, optimizer, X, y, steps=200, E=20):
    losses = []
    for it in tqdm(range(steps)):
        optimizer.zero_grad()
        pY, qF, qU, pU = model(X, E=E)
        ELBO = (pY.log_prob(y)).mean(axis=0).sum()
        ELBO -= torch.sum(distributions.kl_divergence(qU, pU))
        loss = -ELBO
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        print("here")
        
    return losses

In [15]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [16]:
losses = train(model, optimizer, X_train, Y_train, steps=100)

  0%|          | 0/100 [00:00<?, ?it/s]

first


In [ ]:
plt.plot(losses)